# AI_FILL

## Overview
This function automatically fills missing or incomplete data by analyzing patterns from a provided example range. It's particularly useful for completing datasets where there's a predictable relationship between columns, helping to automate data entry and ensure consistency.

## Usage
To use the `AI_FILL` function in Excel, enter it as a formula in a cell, specifying your example and fill ranges, and any optional arguments as needed:

```excel
=AI_FILL(example_range, fill_range, [temperature], [model], [max_tokens])
```

| Parameter      | Type     | Required | Description                                                                                                 | Default |
|---------------|----------|----------|-------------------------------------------------------------------------------------------------------------|---------|
| example_range | 2D list  | Yes      | The range containing complete data that serves as the basis for detecting patterns and relationships.        |         |
| fill_range    | 2D list  | Yes      | The range with incomplete data that will be filled based on the detected patterns from the example_range.    |         |
| temperature   | float    | No       | Controls the randomness/creativity of the response (0.0 to 2.0). Lower values are more deterministic.        | 0.0     |
| model         | string   | No       | The specific AI model ID to use (must support JSON mode, e.g., 'mistral-small-latest').                     | "mistral-small-latest" |
| max_tokens    | int      | No       | Maximum number of tokens for the generated content.                                                         | 1500    |

## Return Value
| Return Value | Type    | Description                                                                                 |
|--------------|---------|---------------------------------------------------------------------------------------------|
| Filled Data  | 2D list | A 2D list with the missing data filled in. Returns `[["Error: ..."]]` on failure.           |

## Limitations
- The quality of the filled data depends on the completeness and representativeness of the example range.
- Large data ranges may exceed model context limits and result in truncated or incomplete responses.
- The function requires an internet connection to access the AI model.
- Model availability and output may vary depending on the provider or API changes.
- Sensitive or confidential data should not be sent to external AI services.
- `temperature` must be a float between 0 and 2 (inclusive). If not, an error message is returned.
- `max_tokens` must be an integer between 5 and 5000 (inclusive). If not, an error message is returned.
- If you hit the API rate limit for your provider, a message is returned instead of raising an exception.

## Benefits
- Automates data completion and cleaning directly in Excel.
- Saves time and improves consistency in data entry and reporting.
- Enables dynamic, context-aware data filling using your own examples.
- More flexible and powerful than manual or native Excel approaches for data completion.

## Examples

### Completing Product Catalog Information
Fill in missing product specifications based on similar products.

**Sample Input Example Range:**
| Product ID | Category | Price | Weight (kg) |
|------------|----------|-------|-------------|
| PRD-001    | Laptop   | 1299  | 1.8         |
| PRD-002    | Laptop   | 999   | 2.1         |
| PRD-003    | Tablet   | 499   | 0.7         |

**Sample Input Fill Range:**
| Product ID | Category | Price | Weight (kg) |
|------------|----------|-------|-------------|
| PRD-004    | Laptop   |       |             |
| PRD-005    | Tablet   |       |             |
| PRD-006    |          | 799   | 1.2         |

```excel
=AI_FILL(A1:D4, A5:D7)
```
**Sample Output:**
| Product ID | Category | Price | Weight (kg) |
|------------|----------|-------|-------------|
| PRD-004    | Laptop   | 1099  | 1.9         |
| PRD-005    | Tablet   | 549   | 0.8         |
| PRD-006    | Laptop   | 799   | 1.2         |

### Filling Employee Information
Complete missing employee department and location information based on job titles.

**Sample Input Example Range:**
| Employee ID | Job Title           | Department | Location    |
|-------------|---------------------|------------|-------------|
| EMP-001     | Sales Manager       | Sales      | New York    |
| EMP-002     | Marketing Specialist| Marketing  | Chicago     |
| EMP-003     | Sales Representative| Sales      | Los Angeles |
| EMP-004     | Software Developer  | Engineering| San Francisco |

**Sample Input Fill Range:**
| Employee ID | Job Title           | Department | Location    |
|-------------|---------------------|------------|-------------|
| EMP-005     | Sales Director      |            |             |
| EMP-006     | UX Designer         |            |             |
| EMP-007     | Marketing Director  |            |             |
| EMP-008     | Senior Developer    |            |             |

```excel
=AI_FILL(A1:D5, A6:D9)
```
**Sample Output:**
| Employee ID | Job Title           | Department | Location    |
|-------------|---------------------|------------|-------------|
| EMP-005     | Sales Director      | Sales      | New York    |
| EMP-006     | UX Designer         | Engineering| San Francisco |
| EMP-007     | Marketing Director  | Marketing  | Chicago     |
| EMP-008     | Senior Developer    | Engineering| San Francisco |

### Completing Financial Forecasts
Fill in missing quarterly projections based on existing data and trends.

**Sample Input Example Range:**
| Metric      | Q1 2024 | Q2 2024 | Q3 2024 | Q4 2024 |
|-------------|---------|---------|---------|---------|
| Revenue     | 250000  | 280000  | 310000  | 350000  |
| Expenses    | 180000  | 195000  | 215000  | 235000  |

**Sample Input Fill Range:**
| Metric      | Q1 2024 | Q2 2024 | Q3 2024 | Q4 2024 |
|-------------|---------|---------|---------|---------|
| Profit      |         |         |         |         |
| Headcount   | 32      | 35      |         |         |

```excel
=AI_FILL(A1:E3, A4:E6)
```
**Sample Output:**
| Metric      | Q1 2024 | Q2 2024 | Q3 2024 | Q4 2024 |
|-------------|---------|---------|---------|---------|
| Profit      | 70000   | 85000   | 95000   | 115000  |
| Headcount   | 32      | 35      | 38      | 42      |

In [ ]:
import requests
import json

def ai_fill(example_range, fill_range, temperature=0.0, model='mistral-small-latest', max_tokens=1500, api_key=None, api_url="https://api.mistral.ai/v1/chat/completions"):
    """
    Uses AI to fill in missing data in a target range by learning patterns from an example range.

    Args:
        example_range (list): 2D list containing complete data as examples for the AI to learn from
        fill_range (list): 2D list containing data with missing values to be filled
        temperature (float, optional): Controls response creativity (0-2). Default is 0.0
        model (str, optional): ID of the model to use
        max_tokens (int, optional): Maximum tokens for response generation. Default is 1500
        api_key (str, optional): API key for authentication (e.g. for Mistral AI)
        api_url (str, optional): OpenAI compatible URL. (e.g., https://api.mistral.ai/v1/chat/completions)

    Returns:
        list: 2D list with missing data filled in, or a 2D list with an error message
    """
    # Validate temperature
    if not isinstance(temperature, (float, int)) or not (0 <= float(temperature) <= 2):
        raise ValueError("temperature must be a float between 0 and 2 (inclusive)")
    # Validate max_tokens
    if not isinstance(max_tokens, int) or not (5 <= max_tokens <= 5000):
        raise ValueError("max_tokens must be an integer between 5 and 5000 (inclusive)")

    # Validate inputs
    if not isinstance(example_range, list) or not example_range:
        return [["Error: Example range is empty or invalid."]]
    if not isinstance(fill_range, list) or not fill_range:
        return [["Error: Fill range is empty or invalid."]]

    # Demo fallback logic (same as ai_ask)
    if api_key is None or api_url is None:
        if 'idToken' in globals():
            api_key = globals()['idToken']
            api_url = "https://llm.boardflare.com"
        else:
            return [["Error: Login on the Functions tab for limited demo usage, or sign up for a free Mistral AI account at https://console.mistral.ai/ and add your own api_key."]]

    # Convert example_range and fill_range to JSON strings for the prompt
    example_json = json.dumps(example_range)
    fill_json = json.dumps(fill_range)

    # Construct a specific prompt for filling data
    fill_prompt = (
        "Fill in the missing values in the target data based on patterns in the example data.\n"
        f"Example data (complete): {example_json}\n\n"
        f"Target data (with missing values): {fill_json}\n\n"
        "Study the patterns in the example data and complete the target data by filling in missing values. Preserve all existing values in the target data."
    )
    fill_prompt += ("\n\nReturn ONLY a JSON object with a key 'items' whose value is a JSON array of arrays (2D array) with the completed target data. "
                    "Do not include any explanatory text, just the JSON object. "
                    "For example: {\"items\": [[\"row1col1\", \"row1col2\"], [\"row2col1\", \"row2col2\"]]}")

    # Prepare the API request payload
    payload = {
        "messages": [{"role": "user", "content": fill_prompt}],
        "temperature": temperature,
        "model": model,
        "max_tokens": max_tokens,
        "response_format": {"type": "json_object"}
    }

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
        "Accept": "application/json"
    }

    try:
        response = requests.post(api_url, headers=headers, json=payload)
        if response.status_code == 429:
            return [["You have hit the rate limit for the API. Please try again later."]]
        response.raise_for_status()
        response_data = response.json()
        content = response_data["choices"][0]["message"]["content"]
        try:
            filled_data = json.loads(content)
            if isinstance(filled_data, dict) and "items" in filled_data:
                filled_data = filled_data["items"]
            elif isinstance(filled_data, dict):
                if "data" in filled_data:
                    filled_data = filled_data["data"]
                elif "filled_data" in filled_data:
                    filled_data = filled_data["filled_data"]
                elif "result" in filled_data:
                    filled_data = filled_data["result"]
            if isinstance(filled_data, list) and all(isinstance(row, list) for row in filled_data):
                if (len(filled_data) == len(fill_range) and all(len(row) == len(fill_range[i]) for i, row in enumerate(filled_data))):
                    return filled_data
                else:
                    return [["Error: AI response dimensions don't match the fill range."]]
            else:
                return [["Error: Unable to parse response. Expected a 2D array."]]
        except (json.JSONDecodeError, ValueError):
            return [["Error: Unable to fill data. The AI response wasn't in the expected format."]]
    except requests.exceptions.RequestException as e:
        return [["Error: API request failed.", str(e)]]

In [ ]:
import ipytest
ipytest.autoconfig()
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent.parent / "test"))
from test_utils import get_graph_token

def inject_id_token():
    token = get_graph_token()
    globals()["idToken"] = token

inject_id_token()

def test_product_catalog():
    example_range = [
        ["Product ID", "Category", "Price", "Weight (kg)"],
        ["PRD-001", "Laptop", 1299, 1.8],
        ["PRD-002", "Laptop", 999, 2.1],
        ["PRD-003", "Tablet", 499, 0.7]
    ]
    fill_range = [
        ["Product ID", "Category", "Price", "Weight (kg)"],
        ["PRD-004", "Laptop", None, None],
        ["PRD-005", "Tablet", None, None],
        ["PRD-006", None, 799, 1.2]
    ]
    result = ai_fill(example_range, fill_range)
    assert isinstance(result, list)
    assert len(result) == len(fill_range)
    assert all(isinstance(row, list) for row in result)

def test_employee_information():
    example_range = [
        ["Employee ID", "Job Title", "Department", "Location"],
        ["EMP-001", "Sales Manager", "Sales", "New York"],
        ["EMP-002", "Marketing Specialist", "Marketing", "Chicago"],
        ["EMP-003", "Sales Representative", "Sales", "Los Angeles"],
        ["EMP-004", "Software Developer", "Engineering", "San Francisco"]
    ]
    fill_range = [
        ["Employee ID", "Job Title", "Department", "Location"],
        ["EMP-005", "Sales Director", None, None],
        ["EMP-006", "UX Designer", None, None],
        ["EMP-007", "Marketing Director", None, None],
        ["EMP-008", "Senior Developer", None, None]
    ]
    result = ai_fill(example_range, fill_range)
    assert isinstance(result, list)
    assert len(result) == len(fill_range)
    assert all(isinstance(row, list) for row in result)

def test_financial_forecasts():
    example_range = [
        ["Metric", "Q1 2024", "Q2 2024", "Q3 2024", "Q4 2024"],
        ["Revenue", 250000, 280000, 310000, 350000],
        ["Expenses", 180000, 195000, 215000, 235000]
    ]
    fill_range = [
        ["Metric", "Q1 2024", "Q2 2024", "Q3 2024", "Q4 2024"],
        ["Profit", None, None, None, None],
        ["Headcount", 32, 35, None, None]
    ]
    result = ai_fill(example_range, fill_range)
    assert isinstance(result, list)
    assert len(result) == len(fill_range)
    assert all(isinstance(row, list) for row in result)

ipytest.run()

In [ ]:
# Gradio Demo
import gradio as gr

def run_ai_fill(example_range, fill_range, temperature, model, max_tokens):
    return ai_fill(example_range, fill_range, temperature=temperature, model=model, max_tokens=max_tokens)

examples = [
    [
        [
            ["Product ID", "Category", "Price", "Weight (kg)"],
            ["PRD-001", "Laptop", 1299, 1.8],
            ["PRD-002", "Laptop", 999, 2.1],
            ["PRD-003", "Tablet", 499, 0.7]
        ],
        [
            ["Product ID", "Category", "Price", "Weight (kg)"],
            ["PRD-004", "Laptop", None, None],
            ["PRD-005", "Tablet", None, None],
            ["PRD-006", None, 799, 1.2]
        ],
        0.0,
        "mistral-small-latest",
        1500
    ],
    [
        [
            ["Employee ID", "Job Title", "Department", "Location"],
            ["EMP-001", "Sales Manager", "Sales", "New York"],
            ["EMP-002", "Marketing Specialist", "Marketing", "Chicago"],
            ["EMP-003", "Sales Representative", "Sales", "Los Angeles"],
            ["EMP-004", "Software Developer", "Engineering", "San Francisco"]
        ],
        [
            ["Employee ID", "Job Title", "Department", "Location"],
            ["EMP-005", "Sales Director", None, None],
            ["EMP-006", "UX Designer", None, None],
            ["EMP-007", "Marketing Director", None, None],
            ["EMP-008", "Senior Developer", None, None]
        ],
        0.0,
        "mistral-small-latest",
        1500
    ],
    [
        [
            ["Metric", "Q1 2024", "Q2 2024", "Q3 2024", "Q4 2024"],
            ["Revenue", 250000, 280000, 310000, 350000],
            ["Expenses", 180000, 195000, 215000, 235000]
        ],
        [
            ["Metric", "Q1 2024", "Q2 2024", "Q3 2024", "Q4 2024"],
            ["Profit", None, None, None, None],
            ["Headcount", 32, 35, None, None]
        ],
        0.0,
        "mistral-small-latest",
        1500
    ]
]

# Set default values to match the first example
first_example = examples[0]

demo = gr.Interface(
    fn=run_ai_fill,
    inputs=[
        gr.Dataframe(label="Example Range", type="array", headers=None, datatype="str", row_count=(1, "dynamic"), col_count=(1, "dynamic"), value=first_example[0]),
        gr.Dataframe(label="Fill Range", type="array", headers=None, datatype="str", row_count=(1, "dynamic"), col_count=(1, "dynamic"), value=first_example[1]),
        gr.Slider(0.0, 2.0, value=first_example[2], step=0.01, label="Temperature"),
        gr.Textbox(value=first_example[3], label="Model"),
        gr.Number(value=first_example[4], label="Max Tokens")
    ],
    outputs=gr.Dataframe(label="Filled Data", type="array"),
    examples=examples,
    description="Automatically fill in missing data in a target range by learning patterns from an example range. Adjust the temperature for creativity, set the maximum number of tokens, and specify the model if desired.",
    flagging_mode="never",
)
demo.launch()